# Adapting OpenAI Advanced Examples for Gemini API

This notebook demonstrates how to adapt advanced examples from the [OpenAI Examples page](https://platform.openai.com/examples) to work with the Gemini API.

**Note:** To use this notebook, you'll need:
1. A Google API key (get one from [Google AI Studio](https://makersuite.google.com/app/apikey))
2. Install the google-generativeai package

## Setup

First, let's install and import the required packages.

In [1]:
!pip install google-generativeai
import google.generativeai as genai

# Configure the Gemini API
genai.configure(api_key="Fake")

# Initialize the model
model = genai.GenerativeModel('gemini-pro')

## Helper Functions

Let's create helper functions to make the Gemini API calls similar to OpenAI's interface.

In [2]:
def get_completion(prompt, temperature=0):
    """Get completion from Gemini API

    Similar to OpenAI's:
    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature
    )
    return response.choices[0].message.content
    """
    response = model.generate_content(
        prompt,
        generation_config={
            "temperature": temperature
        }
    )
    return response.text

def test_prompt(template, test_cases):
    """Test a prompt template with multiple cases"""
    print("Template:\n")
    print(template)
    print("\nTest Cases:\n")
    for case in test_cases:
        print(f"Input: {case}")
        prompt = template.format(text=case)
        result = get_completion(prompt)
        print(f"Output: {result}")
        print("-"*75)

## 1. Few-Shot Learning

Adapting OpenAI's [Few-shot learning example](https://platform.openai.com/examples/default-few-shot).

In [9]:
# Gemini version:
few_shot_template = """
Convert these questions into SQL queries. Follow the examples:

Question: How many users signed up in January?
SQL: SELECT COUNT(*) FROM users WHERE signup_date >= '2024-01-01' AND signup_date < '2024-02-01';

Question: What's the average age of premium users?
SQL: SELECT AVG(age) FROM users WHERE subscription_type = 'premium';

Now convert this question:
{text}
"""

few_shot_test_cases = [
    "Which products have more than 100 reviews?",
    "Find customers who spent over $1000 last month",
    "List the top 5 most active users by post count"
]

def test_prompt(template, test_cases):
    """Test a prompt template with multiple cases"""
    print("Template:\n")
    print(template)
    print("\nTest Cases:\n")
    for case in test_cases:
        print(f"Input: {case}")
        prompt = template.format(text=case)
        result = get_completion(prompt)
        print(f"Output: {result}")
        print("-" * 75)

# Run the test cases
test_prompt(few_shot_template, few_shot_test_cases)


Template:


Convert these questions into SQL queries. Follow the examples:

Question: How many users signed up in January?
SQL: SELECT COUNT(*) FROM users WHERE signup_date >= '2024-01-01' AND signup_date < '2024-02-01';

Question: What's the average age of premium users?
SQL: SELECT AVG(age) FROM users WHERE subscription_type = 'premium';

Now convert this question:
{text}


Test Cases:

Input: Which products have more than 100 reviews?
Output: SQL: SELECT product_id, COUNT(*) AS num_reviews FROM reviews GROUP BY product_id HAVING num_reviews > 100;
---------------------------------------------------------------------------
Input: Find customers who spent over $1000 last month
Output: SQL: SELECT * FROM customers WHERE total_spent_last_month > 1000;
---------------------------------------------------------------------------
Input: List the top 5 most active users by post count
Output: SQL: SELECT username, COUNT(*) AS post_count FROM posts GROUP BY username ORDER BY post_count DESC LI

## 2. Chain of Thought Reasoning

Adapting OpenAI's [Chain-of-thought example](https://platform.openai.com/examples/default-chain-of-thought).

In [5]:
# OpenAI version:
"""
client = OpenAI()
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{
        "role": "user",
        "content": f"Let's solve this step by step: {text}"
    }]
)
print(response.choices[0].message.content)
"""

# Gemini version:
cot_template = """
Let's solve this problem step by step:

Problem: {text}

Follow this reasoning process:
1. Identify the key information and variables
2. Break down the problem into smaller parts
3. Solve each part systematically
4. Combine the results
5. Verify the solution

Solution:
"""

cot_test_cases = [
    """
    A store has a 20% off sale. Additionally, members get an extra 10% off the
    sale price. If a member buys an item originally priced at $150, how much
    will they pay after both discounts?
    """,
    """
    A train travels 150 km at 60 km/h, then another 200 km at 100 km/h.
    What is the average speed for the entire journey?
    """
]

test_prompt(cot_template, cot_test_cases)

Template:


Let's solve this problem step by step:

Problem: {text}

Follow this reasoning process:
1. Identify the key information and variables
2. Break down the problem into smaller parts
3. Solve each part systematically
4. Combine the results
5. Verify the solution

Solution:


Test Cases:

Input: 
    A store has a 20% off sale. Additionally, members get an extra 10% off the 
    sale price. If a member buys an item originally priced at $150, how much 
    will they pay after both discounts?
    
Output: **1. Identify the key information and variables**

* Original price of the item: $150
* Sale discount: 20%
* Member discount: 10%
* Final price: x

**2. Break down the problem into smaller parts**

* Calculate the sale price: $150 - (20% of $150) = $120
* Calculate the member discount: 10% of $120 = $12
* Calculate the final price: $120 - $12 = $108

**3. Solve each part systematically**

* Sale price: $150 - (20% of $150) = $150 - $30 = $120
* Member discount: 10% of $120 = $12


## 3. Code Generation and Analysis

Adapting OpenAI's [Code generation example](https://platform.openai.com/examples/default-code-generation).

In [6]:
# OpenAI version:
"""
client = OpenAI()
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{
        "role": "user",
        "content": f"Write a Python function that: {text}"
    }]
)
print(response.choices[0].message.content)
"""

# Gemini version:
code_template = """
Write Python code for the following task. Include:
1. Complete implementation
2. Docstring with explanation
3. Example usage
4. Error handling
5. Performance considerations

Task: {text}

Response format:
```python
# Implementation
[code here]

# Example usage
[example code]
```
"""

code_test_cases = [
    "Create a function that finds the longest palindrome in a string",
    "Implement a cache decorator with timeout functionality",
    "Create a class for handling API rate limiting with exponential backoff"
]

test_prompt(code_template, code_test_cases)

Template:


Write Python code for the following task. Include:
1. Complete implementation
2. Docstring with explanation
3. Example usage
4. Error handling
5. Performance considerations

Task: {text}

Response format:
```python
# Implementation
[code here]

# Example usage
[example code]
```


Test Cases:

Input: Create a function that finds the longest palindrome in a string
Output: ```python
def longest_palindrome(string):
  """Finds the longest palindrome in a string.

  A palindrome is a string that reads the same forwards and backwards.

  Args:
    string: The string to search for the longest palindrome in.

  Returns:
    The longest palindrome in the string.

  Raises:
    TypeError: If the input string is not a string.
  """

  # Check if the input is a string.
  if not isinstance(string, str):
    raise TypeError("Input must be a string.")

  # If the string is empty, return an empty string.
  if not string:
    return ""

  # Initialize the longest palindrome to the first cha

## 4. Complex Data Extraction

Adapting OpenAI's [Data extraction example](https://platform.openai.com/examples/default-extraction).

In [7]:
# OpenAI version:
"""
client = OpenAI()
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{
        "role": "user",
        "content": f"Extract structured data from this text and format as JSON: {text}"
    }]
)
print(response.choices[0].message.content)
"""

# Gemini version:
extraction_template = """
Extract structured information from this text and format as JSON:

Text: {text}

Extract:
1. Events (with dates)
2. People (with roles)
3. Organizations
4. Locations
5. Key metrics/numbers
6. Important relationships

Format as valid JSON with appropriate nesting.
"""

extraction_test_cases = [
    """
    In Q3 2023, Tesla CEO Elon Musk announced a partnership with Toyota Motor
    Corporation to develop new battery technology. The $5 billion joint venture,
    based in Austin, Texas, aims to begin production by 2025. Toyota President
    Koji Sato stated the collaboration would create 3,000 new jobs. The companies
    also plan to build a new research facility in Osaka, Japan.
    """,
    """
    Microsoft's acquisition of Activision Blizzard for $68.7 billion was approved
    by EU regulators on March 15, 2024. CEO Satya Nadella announced the deal at
    their Redmond headquarters, joined by Activision CEO Bobby Kotick. The merger
    will affect 10,000 employees across offices in California, London, and Shanghai.
    """
]

test_prompt(extraction_template, extraction_test_cases)

Template:


Extract structured information from this text and format as JSON:

Text: {text}

Extract:
1. Events (with dates)
2. People (with roles)
3. Organizations
4. Locations
5. Key metrics/numbers
6. Important relationships

Format as valid JSON with appropriate nesting.


Test Cases:

Input: 
    In Q3 2023, Tesla CEO Elon Musk announced a partnership with Toyota Motor 
    Corporation to develop new battery technology. The $5 billion joint venture, 
    based in Austin, Texas, aims to begin production by 2025. Toyota President 
    Koji Sato stated the collaboration would create 3,000 new jobs. The companies 
    also plan to build a new research facility in Osaka, Japan.
    
Output: ```json
{
  "events": [
    {
      "date": "Q3 2023",
      "description": "Tesla CEO Elon Musk announced a partnership with Toyota Motor Corporation to develop new battery technology."
    },
    {
      "date": "2025",
      "description": "The joint venture aims to begin production."
    }
  ],


## 5. Complex Reasoning

Adapting OpenAI's [Complex reasoning example](https://platform.openai.com/examples/default-reasoning).

In [8]:
# OpenAI version:
"""
client = OpenAI()
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{
        "role": "user",
        "content": f"Analyze this scenario and provide recommendations: {text}"
    }]
)
print(response.choices[0].message.content)
"""

# Gemini version:
reasoning_template = """
Analyze this business scenario using systematic reasoning:

Scenario: {text}

Provide:
1. Situation Analysis
   - Current state
   - Key challenges
   - Market context
   - Stakeholders

2. Options Analysis
   - Potential solutions
   - Pros and cons
   - Resource requirements
   - Implementation complexity

3. Recommendations
   - Short-term actions
   - Long-term strategy
   - Risk mitigation
   - Success metrics
"""

reasoning_test_cases = [
    """
    A software company is deciding whether to pivot their main product from a
    desktop application to a cloud-based SaaS solution. They have 100,000 existing
    customers, $10M annual revenue, and 50 employees. The cloud transition would
    take 18 months and cost $5M. Market research shows growing demand for cloud
    solutions but strong competition.
    """,
    """
    A retail chain is experiencing 20% decline in foot traffic across their 50 stores
    due to increasing online shopping trends. They're considering either investing
    $20M in e-commerce infrastructure or converting stores into hybrid
    fulfillment centers. They have 6 months of runway at current burn rate.
    """
]

test_prompt(reasoning_template, reasoning_test_cases)

Template:


Analyze this business scenario using systematic reasoning:

Scenario: {text}

Provide:
1. Situation Analysis
   - Current state
   - Key challenges
   - Market context
   - Stakeholders

2. Options Analysis
   - Potential solutions
   - Pros and cons
   - Resource requirements
   - Implementation complexity

3. Recommendations
   - Short-term actions
   - Long-term strategy
   - Risk mitigation
   - Success metrics


Test Cases:

Input: 
    A software company is deciding whether to pivot their main product from a 
    desktop application to a cloud-based SaaS solution. They have 100,000 existing 
    customers, $10M annual revenue, and 50 employees. The cloud transition would 
    take 18 months and cost $5M. Market research shows growing demand for cloud 
    solutions but strong competition.
    
Output: **1. Situation Analysis**

**Current State:**
- 100,000 existing customers
- $10M annual revenue
- 50 employees
- Desktop application as the main product

**Key Challeng